In [20]:
import pandas as pd

In [8]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold


# Se levanta el set de entrenamiento generado

In [9]:
RF_train = pd.read_csv("RF_train")
y_train = pd.read_csv("y_train")

In [10]:
model_RF = RandomForestRegressor(n_jobs = -1, n_estimators = 200)

In [11]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [160, 180, 200],
    'max_features': [0.2, 0.4, 0.8, 1],
    'min_samples_leaf': [3, 4, 6, 8],
    'min_samples_split': [6, 8, 10, 12, 14, 16],
}

In [12]:
randomized = RandomizedSearchCV(model_RF, param_grid, n_iter = 10, verbose = 1, n_jobs = -1)
randomized.fit(RF_train, y_train.values.ravel())

F:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 34.1min finished


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'bootstrap': [True], 'max_depth': [40, 80, 120, 160], 'max_features': [0.2, 0.4, 0.8, 1], 'min_samples_leaf': [3, 4, 6, 8], 'min_samples_split': [6, 8, 10, 12, 14, 16]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [13]:
randomized.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=160,
           max_features=0.4, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=4, min_samples_split=12,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [22]:
pd.DataFrame(data = randomized.cv_results_).sort_values(by =["rank_test_score"]).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_split,param_min_samples_leaf,param_max_features,param_max_depth,param_bootstrap,params,...,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
7,1069.653074,3.280172,1.112422,0.224359,12,4,0.4,160,True,"{'min_samples_split': 12, 'min_samples_leaf': ...",...,0.783535,0.782423,0.782339,0.001013,1,0.924467,0.923682,0.923658,0.923936,0.000376
0,1712.879426,6.165824,11.345142,0.537411,14,4,0.4,80,True,"{'min_samples_split': 14, 'min_samples_leaf': ...",...,0.783572,0.782266,0.782227,0.001114,2,0.919205,0.918634,0.918594,0.918811,0.000279
1,851.824536,4.258960,11.804071,0.597727,10,4,0.2,120,True,"{'min_samples_split': 10, 'min_samples_leaf': ...",...,0.783496,0.782234,0.782125,0.001167,3,0.924459,0.924011,0.924141,0.924204,0.000188
9,713.404363,31.557437,4.960628,1.816463,16,3,0.2,160,True,"{'min_samples_split': 16, 'min_samples_leaf': ...",...,0.783470,0.782088,0.782102,0.001111,4,0.914211,0.913647,0.913804,0.913887,0.000238
2,872.157558,14.063550,15.446446,2.358988,12,4,0.2,40,True,"{'min_samples_split': 12, 'min_samples_leaf': ...",...,0.783412,0.782229,0.782090,0.001140,5,0.918339,0.917789,0.917824,0.917984,0.000251


In [14]:
randomized.best_params_

{'min_samples_split': 12,
 'min_samples_leaf': 4,
 'max_features': 0.4,
 'max_depth': 160,
 'bootstrap': True}